In [1]:
import pandas as pd
import requests
from datetime import datetime, timedelta
# from datetime import datetime,timedelta
import numpy as np
from get7DMA import get_7dma_state

In [2]:
def col_check_state_raw_csv(df):
    cols_ls=df.columns
    reqd_col_ls=["Date",
                 "State/UTCode",
                 "deltaConfirmedForState",
                 "deltaDeceasedForState",
                 "deltaRecoveredForState",
                 "deltaTestedForState",
                 "deltaVaccinated1ForState",
                 "deltaVaccinated2ForState",
                 "delta21_14confirmedForState",
                 "7DmaConfirmedForState",
                 "7DmaDeceasedForState",
                 "7DmaRecoveredForState",
                 "7DmaTestedForState",
                 "7DmaVaccinated1ForState",
                 "7DmaVaccinated2ForState",
                 "District",
                 "deltaConfirmedForDistrict",
                 "deltaDeceasedForDistrict",
                 "deltaRecoveredForDistrict",
                 "deltaTestedForDistrict",
                 "deltaVaccinated1ForDistrict",
                 "deltaVaccinated2ForDistrict",
                 "delta21_14confirmedForDistrict",
                 "7DmaConfirmedForDistrict",
                 "7DmaDeceasedForDistrict",
                 "7DmaRecoveredForDistrict",
                 "7DmaTestedForDistrict",
                 "7DmaVaccinated1ForDistrict",
                 "7DmaVaccinated2ForDistrict",
                 "districtPopulation",
                 "tested_last_updated_district",
                 "tested_source_district",
                 "notesForDistrict",
                 "cumulativeConfirmedNumberForDistrict",
                 "cumulativeDeceasedNumberForDistrict",
                 "cumulativeRecoveredNumberForDistrict",
                 "cumulativeTestedNumberForDistrict",
                 "cumulativeVaccinated1NumberForDistrict",
                 "cumulativeVaccinated2NumberForDistrict",
                 "last_updated",
                 "statePopulation",
                 "tested_last_updated_state",
                 "tested_source_state",
                 "notesForState",
                 "cumulativeConfirmedNumberForState",
                 "cumulativeDeceasedNumberForState",
                 "cumulativeRecoveredNumberForState",
                 "cumulativeTestedNumberForState",
                 "cumulativeVaccinated1NumberForState",
                 "cumulativeVaccinated2NumberForState"
                ]
    for col in reqd_col_ls:
        if col not in cols_ls:
            df[col]=0
    df_formatted=df[reqd_col_ls]
    return df_formatted
            

In [3]:
#https://github.com/covid19india/covid19india-react/blob/master/src/constants.js#L398
STATE_NAMES = {
  'AP': 'Andhra Pradesh',
  'AR': 'Arunachal Pradesh',
  'AS': 'Assam',
  'BR': 'Bihar',
  'CT': 'Chhattisgarh',
  'GA': 'Goa',
  'GJ': 'Gujarat',
  'HR': 'Haryana',
  'HP': 'Himachal Pradesh',
  'JH': 'Jharkhand',
  'KA': 'Karnataka',
  'KL': 'Kerala',
  'MP': 'Madhya Pradesh',
  'MH': 'Maharashtra',
  'MN': 'Manipur',
  'ML': 'Meghalaya',
  'MZ': 'Mizoram',
  'NL': 'Nagaland',
  'OR': 'Odisha',
  'PB': 'Punjab',
  'RJ': 'Rajasthan',
  'SK': 'Sikkim',
  'TN': 'Tamil Nadu',
  'TG': 'Telangana',
  'TR': 'Tripura',
  'UT': 'Uttarakhand',
  'UP': 'Uttar Pradesh',
  'WB': 'West Bengal',
  'AN': 'Andaman and Nicobar Islands',
  'CH': 'Chandigarh',
  'DN': 'Dadra and Nagar Haveli and Daman and Diu',
  'DL': 'Delhi',
  'JK': 'Jammu and Kashmir',
  'LA': 'Ladakh',
  'LD': 'Lakshadweep',
  'PY': 'Puducherry',
  # 'TT': 'India',
 # [UNASSIGNED_STATE_CODE]: 'Unassigned',
}

In [4]:
# cowin_codes=pd.read_csv("../CSV/CowinAppStateAndDistrictCode.csv")
# cowin_codes.head()

In [5]:
#Creating the mapping dictionary for multiple districts addition scenario
cowin_codes=pd.read_csv("../CSV/CowinAppStateAndDistrictCode.csv")
cowin_codes_map={}
cowin_codes["district_id_add"]=cowin_codes[["district_id_add"]].fillna("NULL")

for i,d_id in enumerate(cowin_codes["district_id"]):
#     print(d_id)
    add_dist_id=cowin_codes.loc[i,"district_id_add"]
    if add_dist_id !="NULL":
        cowin_codes_map[str(int(d_id))]=[str(int(add_dist_id)),str(int(d_id))]
    else:
        cowin_codes_map[str(int(d_id))]=[str(int(d_id))]
# print(cowin_codes_map)

In [6]:
cowin_codes_map
#cowin_codes

{'3': ['3'],
 '1': ['1'],
 '2': ['2'],
 '9': ['9'],
 '10': ['10'],
 '11': ['11'],
 '5': ['5'],
 '4': ['4'],
 '7': ['7'],
 '12': ['12'],
 '13': ['13'],
 '14': ['14'],
 '8': ['8'],
 '15': ['15'],
 '16': ['16'],
 '6': ['6'],
 '22': ['22'],
 '20': ['20'],
 '25': ['25'],
 '23': ['23'],
 '42': ['42'],
 '17': ['39', '17'],
 '24': ['24'],
 '27': ['27'],
 '21': ['21'],
 '33': ['33'],
 '29': ['29'],
 '40': ['40'],
 '31': ['31'],
 '18': ['18'],
 '32': ['32'],
 '36': ['36'],
 '19': ['19'],
 '39': ['17', '39'],
 '35': ['35'],
 '37': ['37'],
 '30': ['30'],
 '26': ['26'],
 '34': ['34'],
 '41': ['41'],
 '28': ['28'],
 '38': ['38'],
 '46': ['46'],
 '47': ['47'],
 '765': ['765'],
 '57': ['57'],
 '66': ['66'],
 '766': ['766'],
 '58': ['58'],
 '48': ['48'],
 '62': ['62'],
 '59': ['59'],
 '43': ['43'],
 '67': ['67'],
 '60': ['60'],
 '53': ['53'],
 '68': ['68'],
 '764': ['764'],
 '54': ['54'],
 '49': ['49'],
 '50': ['50'],
 '51': ['51'],
 '69': ['69'],
 '61': ['61'],
 '63': ['63'],
 '767': ['767'],
 '55': [

In [7]:
def vaccination_numbers_api(state_name,Date):
    cowin_codes=pd.read_csv("../CSV/CowinAppStateAndDistrictCode.csv")
    
    if state_name=="Dadra and Nagar Haveli and Daman and Diu":
        vaccination_number_DNH=vaccination_numbers_api("Dadra and Nagar Haveli",Date)
        vaccination_number_DND=vaccination_numbers_api("Daman and Diu",Date)
        
        state_cumulative_vaccinated1=vaccination_number_DNH[0]+vaccination_number_DND[0]
        state_cumulative_vaccinated2=vaccination_number_DNH[1]+vaccination_number_DND[1]
        state_cumulative_vaccinated=vaccination_number_DNH[2]+vaccination_number_DND[2]
        district_vacc_dict=vaccination_number_DNH[3]
        district_vacc_dict.update(vaccination_number_DND[3])
        
        return(state_cumulative_vaccinated1,state_cumulative_vaccinated2,state_cumulative_vaccinated,district_vacc_dict)
    
#     print(state_name)
    district_cumulative_vaccinated1=0
    district_cumulative_vaccinated2=0
    district_cumulative_vaccinated=0
    
    state_cumulative_vaccinated1=0
    state_cumulative_vaccinated2=0
    state_cumulative_vaccinated=0
        
    #State Numbers
    #State Numbers
    if state_name=="Dadra and Nagar Haveli":
        cowin_state_code=8
    elif state_name=="Daman and Diu":
        cowin_state_code=37
    else:
        cowin_state_code=cowin_codes.loc[cowin_codes["state_name"]==state_name,"state_id"]
        cowin_state_code.reset_index(inplace=True,drop=True)
        cowin_state_code=list(set(cowin_state_code))[0]
    
    
    api_url_state="https://api.cowin.gov.in/api/v1/reports/v2/getPublicReports?state_id="+str(cowin_state_code)+"&date="+Date
#     print(api_url_state)
    api_data_state=requests.get(api_url_state)
#     print(api_data_state.json())
    
    state_cumulative_vaccinated1=api_data_state.json()["topBlock"]["vaccination"]["tot_dose_1"]
    state_cumulative_vaccinated2=api_data_state.json()["topBlock"]["vaccination"]["tot_dose_2"]
    state_cumulative_vaccinated=api_data_state.json()["topBlock"]["vaccination"]["total"]
    
    district_vacc_dict={}
    for ele in api_data_state.json()["getBeneficiariesGroupBy"]:
        district_vacc_dict[ele["district_id"]]=[ele["partial_vaccinated"],ele["totally_vaccinated"],ele["total"]]
    
    return (state_cumulative_vaccinated1,state_cumulative_vaccinated2,state_cumulative_vaccinated,district_vacc_dict)

In [8]:
# vaccination_numbers_api("Karnataka","2021-12-02")[3]

In [9]:
import json
import traceback
from datetime import datetime
import time

def addLogging(logDict:dict):
    loggingsFile = '../log.json'

    with open(loggingsFile) as f:
        data = json.load(f)
        #data = []

    data.append(logDict)

    with open(loggingsFile, 'w') as f:
        json.dump(data, f)

def currentTimeUTC(date):
    # return time.mktime(datetime.strptime(datetime.now().strftime('%d/%m/%Y'),'%d/%m/%Y').timetuple())
    return int(time.mktime(datetime.strptime(date,'%Y-%m-%d').timetuple()))
    #return datetime.now().strftime('%d/%m/%Y %H:%M:%S')

def updateJSONLog(stateName,date):
    addLogging({
      "update": stateName + ":\n No district level updates received in state government bulletin",
      "timestamp": currentTimeUTC(date)
   })

def removeLogging(date):
    loggingsFile = "../log.json"
    
    with open(loggingsFile) as f:
        data = json.load(f)

    data = list(filter(lambda i: i['timestamp'] != time.mktime(datetime.strptime(date,'%Y-%m-%d').timetuple()), data))

    with open(loggingsFile, 'w') as f:
        json.dump(data, f)

    # State totals for confirmed cases reported on https://www.mygov.in/covid-19 (xxx) different from district total for confirmed cases reported in state government bulletin (yyy)
# try:
#     print(5/0)
# except ZeroDivisionError:
#     fullTraceback = str(traceback.format_exc())
#     addLogging({'timestamp': currentTimeUTC(), 'level': 'error', 'traceback': fullTraceback})

In [10]:
def updateDerivedValues(StateCode,Date):
    cowin_codes=pd.read_csv("../CSV/CowinAppStateAndDistrictCode.csv")
    sources=pd.read_csv("../sources.csv") 
    try:
        state_raw_csv=pd.read_csv(f"../RAWCSV/{Date}/{StateCode}_raw.csv",index_col=False)
    except:
        state_raw_csv=pd.read_csv(f"../RAWCSV/{Date}/myGov/{StateCode}_raw.csv",index_col=False)
        if (sources[sources["StateCode"] == StateCode]["myGov"] != "yes").item():
            updateJSONLog((sources[sources["StateCode"] == StateCode]["StateName"]).item(),Date)
            # print("yes")
        # RAWCSV/2021-11-05/myGov/AN_raw.csv
    state_population=pd.read_csv("../CSV/StatePopulation.csv")
    district_population=pd.read_csv("../CSV/DistrictPopulation.csv")
    
    #Check and insert cummulativeDeceasedState 
    # if state_raw_csv["cumulativeDeceasedNumberForState"].sum() == 0:
    #     if state_raw_csv["cumulativeDeceasedNumberForDistrict"].sum() != 0:
    #         state_raw_csv["cumulativeDeceasedNumberForState"] = state_raw_csv["cumulativeDeceasedNumberForDistrict"].sum()
    state_raw_csv=col_check_state_raw_csv(state_raw_csv)
    
    state_name=STATE_NAMES[StateCode]
    
    print("\n")
    print(state_name)
    print("---------------------------------")
    
    state_raw_csv["State/UTCode"]=StateCode
    
    # if StateCode == "DN":
    #     return None
    # #gunjan will provide
    # else:
    vaccination_numbers=vaccination_numbers_api(state_name,Date)
    
    state_cumulative_vaccinated1=vaccination_numbers[0]
    state_cumulative_vaccinated2=vaccination_numbers[1]
    state_cumulative_vaccinated=vaccination_numbers[2]
    
    state_raw_csv.loc[state_raw_csv["State/UTCode"]==StateCode,"cumulativeVaccinated1NumberForState"]=state_cumulative_vaccinated1
    state_raw_csv.loc[state_raw_csv["State/UTCode"]==StateCode,"cumulativeVaccinated2NumberForState"]=state_cumulative_vaccinated2
    state_raw_csv.loc[state_raw_csv["State/UTCode"]==StateCode,"cumulativeVaccinatedNumberForState"]=state_cumulative_vaccinated
    
    #Reading State population numbers from static source
    state_population_number=state_population.loc[state_population["State"]==StateCode,"StatePop"]
    state_population_number.reset_index(inplace=True,drop=True)
    state_population_number=state_population_number[0]    
    state_raw_csv.loc[state_raw_csv["State/UTCode"]==StateCode,"statePopulation"]=state_population_number
    
    #Deriving the delta numbers
    
    prev_date=datetime.strptime(Date,"%Y-%m-%d")+timedelta(days=(-1))
    prev_date_str=str(prev_date.date())
    try:
        previous_state_raw_csv=pd.read_csv(f"../RAWCSV/{prev_date_str}/{StateCode}_raw.csv")
    except:
        previous_state_raw_csv=pd.read_csv(f"../RAWCSV/{prev_date_str}/myGov/{StateCode}_raw.csv")
    
    vaccination_numbers_yesterday=vaccination_numbers_api(state_name,prev_date_str)
    
    try:
        if state_raw_csv["cumulativeConfirmedNumberForState"].sum() != 0:
            state_raw_csv["deltaConfirmedForState"]=state_raw_csv["cumulativeConfirmedNumberForState"]-previous_state_raw_csv["cumulativeConfirmedNumberForState"]
    except:
        state_raw_csv["deltaConfirmedForState"] = None
    
    try:
        if state_raw_csv["cumulativeDeceasedNumberForState"].sum() != 0:
            state_raw_csv["deltaDeceasedForState"]=state_raw_csv["cumulativeDeceasedNumberForState"]-previous_state_raw_csv["cumulativeDeceasedNumberForState"]
    except:
        state_raw_csv["deltaDeceasedForState"] = None
        
    try:    
        if state_raw_csv["cumulativeRecoveredNumberForState"].sum() != 0:
            state_raw_csv["deltaRecoveredForState"]=state_raw_csv["cumulativeRecoveredNumberForState"]-previous_state_raw_csv["cumulativeRecoveredNumberForState"]
    except:
        state_raw_csv["deltaDeceasedForState"] = None
    
    try:
        if state_raw_csv["cumulativeVaccinatedNumberForState"].sum() != 0:
            state_raw_csv["deltaVaccinatedForState"]=state_raw_csv["cumulativeVaccinatedNumberForState"][0]-vaccination_numbers_yesterday[2]
    except:
        state_raw_csv["deltaVaccinatedForState"] = None
        
    try:    
        if state_raw_csv["cumulativeVaccinated1NumberForState"].sum() != 0:
            state_raw_csv["deltaVaccinated1ForState"]=state_raw_csv["cumulativeVaccinated1NumberForState"]-vaccination_numbers_yesterday[0]
    except:
        state_raw_csv["deltaVaccinated1ForState"] =None
    
    try:
        if state_raw_csv["cumulativeVaccinated2NumberForState"].sum() != 0:
            state_raw_csv["deltaVaccinated2ForState"]=state_raw_csv["cumulativeVaccinated2NumberForState"]-vaccination_numbers_yesterday[1]
    except:
        state_raw_csv["deltaVaccinated2ForState"] = None
    
    try:        
        if state_raw_csv["cumulativeTestedNumberForState"].sum() != 0:
            state_raw_csv["deltaTestedForState"]=state_raw_csv["cumulativeTestedNumberForState"].astype("int64")-previous_state_raw_csv["cumulativeTestedNumberForState"].astype("int64")
    except FileNotFoundError:
        print("Tested Error")
        state_raw_csv["deltaTestedForState"] = None
        
#     state_raw_csv.loc[:,"7DmaConfirmedForState"]=0
#     state_raw_csv.loc[:,"7DmaDeceasedForState"]=0
#     state_raw_csv.loc[:,"7DmaRecoveredForState"]=0
#     state_raw_csv.loc[:,"7DmaVaccinatedForState"]=0
#     state_raw_csv.loc[:,"7DmaTestedForState"]=0
#     state_raw_csv.loc[:,"7DmaVaccinated1ForState"]=0
#     state_raw_csv.loc[:,"7DmaVaccinated2ForState"]=0
    
#     try:
#         avg_counter = 0
#         for i in range(7):
#             prev_date=datetime.strptime(Date,"%Y-%m-%d")+timedelta(days=(-1*i))
#             # print(str(prev_date))
#             prev_date_str=str(prev_date.date())
#             if i == 0:
#                 previous_state_raw_csv = state_raw_csv
#             else:
#                 try:
#                     previous_state_raw_csv=pd.read_csv(f"../RAWCSV/{prev_date_str}/{StateCode}_raw.csv")
#                 except:
#                     previous_state_raw_csv=pd.read_csv(f"../RAWCSV/{prev_date_str}/myGov/{StateCode}_raw.csv")
                
#             #state_raw_csv["7DmaConfirmedForState"]+=int(previous_state_raw_csv["cumulativeConfirmedNumberForState"][0])
#             #state_raw_csv["7DmaDeceasedForState"]+=int(previous_state_raw_csv["cumulativeDeceasedNumberForState"][0])
#             state_raw_csv["7DmaConfirmedForState"]+=previous_state_raw_csv["cumulativeConfirmedNumberForState"][0]
#             state_raw_csv["7DmaDeceasedForState"]+=previous_state_raw_csv["cumulativeDeceasedNumberForState"][0]
#             state_raw_csv["7DmaRecoveredForState"]+=previous_state_raw_csv["cumulativeRecoveredNumberForState"][0]
#             state_raw_csv["7DmaVaccinatedForState"]+=vaccination_numbers[2]
#             state_raw_csv["7DmaTestedForState"]+=previous_state_raw_csv["cumulativeTestedNumberForState"][0]
#             state_raw_csv["7DmaVaccinated1ForState"]+=vaccination_numbers[0]
#             state_raw_csv["7DmaVaccinated2ForState"]+=vaccination_numbers[1]
#             avg_counter += 1

            
#         state_raw_csv["7DmaConfirmedForState"]=(state_raw_csv["7DmaConfirmedForState"]/7)#.astype('int64')
#         state_raw_csv["7DmaDeceasedForState"]=(state_raw_csv["7DmaDeceasedForState"]/7)#.astype('int64')
#         state_raw_csv["7DmaRecoveredForState"]=round(state_raw_csv["7DmaRecoveredForState"]/7,0)
#         state_raw_csv["7DmaVaccinatedForState"]=round(state_raw_csv["7DmaVaccinatedForState"]/7,0)
#         state_raw_csv["7DmaTestedForState"]=round(state_raw_csv["7DmaTestedForState"]/7,0)
#         state_raw_csv["7DmaVaccinated1ForState"]=round(state_raw_csv["7DmaVaccinated1ForState"]/7,0)
#         state_raw_csv["7DmaVaccinated2ForState"]=round(state_raw_csv["7DmaVaccinated2ForState"]/7,0)
#     except:
#         raise
#         print("Error:"+state_name)
        
        
        
    for district in state_raw_csv["District"]:
        # print(district,state_name)
        cowin_district_code=cowin_codes.loc[((cowin_codes["district_name"]==district)&(cowin_codes["state_name"]==state_name)),"district_id"]
        cowin_district_code.reset_index(inplace=True,drop=True)
        try:
            cowin_district_code=cowin_district_code[0]
        except IndexError:
            # print(f"{district} not found in Cowin States Districts Code")
            district_vaccination_data=False
            pass
        except KeyError:
            # print(f"{district} not found in Cowin States Districts Code")
            district_vaccination_data=False
            pass
        else:
            ls_districts_add=cowin_codes_map[str(cowin_district_code)]
            district_cumulative_vaccinated1=0
            district_cumulative_vaccinated2=0
            district_cumulative_vaccinated=0
            for dist in ls_districts_add:
                district_cumulative_vaccinated1+=vaccination_numbers[3][dist][0]
                district_cumulative_vaccinated2+=vaccination_numbers[3][dist][1]
                district_cumulative_vaccinated+=vaccination_numbers[3][dist][2]
                district_vaccination_data=True
                
        if district_vaccination_data:
            state_raw_csv.loc[((state_raw_csv["State/UTCode"]==StateCode)&(state_raw_csv["District"]==district)),"cumulativeVaccinated1NumberForDistrict"]=district_cumulative_vaccinated1
            state_raw_csv.loc[((state_raw_csv["State/UTCode"]==StateCode)&(state_raw_csv["District"]==district)),"cumulativeVaccinated2NumberForDistrict"]=district_cumulative_vaccinated2
            state_raw_csv.loc[((state_raw_csv["State/UTCode"]==StateCode)&(state_raw_csv["District"]==district)),"cumulativeVaccinatedNumberForDistrict"]=district_cumulative_vaccinated

        
        district_population_number=district_population.loc[((district_population["State"]==StateCode) & (district_population["District"]==district)) ,"DistrictPop"]#[0]
        district_population_number.reset_index(inplace=True,drop=True)
        try:
            district_population_number=district_population_number[0]
        except IndexError:
            print(f"{district} population numbers not found")
            pass
        except KeyError:
            print(f"{district} population numbers not found")
            pass
        else:
            state_raw_csv.loc[((state_raw_csv["State/UTCode"]==StateCode)&(state_raw_csv["District"]==district)),"districtPopulation"]=district_population_number
    
    #delta for district
#         prev_date=datetime.strptime(Date,"%Y-%m-%d")+timedelta(days=(-1))
#         prev_date_str=str(prev_date.year)+"-"+str(prev_date.month)+"-"+str(prev_date.day)
#         previous_state_raw_csv=pd.read_csv(f"../RAWCSV/{prev_date_str}/{StateCode}_raw.csv")
        
#         if state_raw_csv["cumulativeConfirmedNumberForDistrict"].sum() != 0:
#             state_raw_csv.loc[state_raw_csv["District"] == district,"deltaConfirmedForDistrict"]=state_raw_csv.loc[state_raw_csv["District"] == district,"cumulativeConfirmedNumberForDistrict"].item()-previous_state_raw_csv.loc[previous_state_raw_csv["District"] == district,"cumulativeConfirmedNumberForDistrict"].item()

#         if state_raw_csv["cumulativeDeceasedNumberForDistrict"].sum() != 0:
#             state_raw_csv.loc[state_raw_csv["District"] == district,"deltaDeceasedForDistrict"]=state_raw_csv.loc[state_raw_csv["District"] == district,"cumulativeDeceasedNumberForDistrict"].item()-previous_state_raw_csv.loc[previous_state_raw_csv["District"] == district,"cumulativeDeceasedNumberForDistrict"].item()

#         if state_raw_csv["cumulativeRecoveredNumberForDistrict"].sum() != 0:
#             state_raw_csv.loc[state_raw_csv["District"] == district,"deltaRecoveredForDistrict"]=state_raw_csv.loc[state_raw_csv["District"] == district,"cumulativeRecoveredNumberForDistrict"].item()-previous_state_raw_csv.loc[previous_state_raw_csv["District"]  == district,"cumulativeRecoveredNumberForDistrict"].item()

#         if state_raw_csv["cumulativeTestedNumberForDistrict"].sum() != 0:
#             state_raw_csv.loc[state_raw_csv["District"] == district,"deltaTestedForDistrict"]=state_raw_csv.loc[state_raw_csv["District"] == district,"cumulativeTestedNumberForDistrict"].item()-previous_state_raw_csv.loc[previous_state_raw_csv["District"] == district,"cumulativeTestedNumberForDistrict"].item()

#         if state_raw_csv["cumulativeVaccinatedNumberForDistrict"].sum() != 0:
#             state_raw_csv.loc[state_raw_csv["District"] == district,"deltaVaccinatedForDistrict"]=state_raw_csv.loc[state_raw_csv["District"] == district,"cumulativeVaccinatedNumberForDistrict"].item()-previous_state_raw_csv.loc[previous_state_raw_csv["District"] == district,"cumulativeVaccinatedNumberForDistrict"].item()

#         if state_raw_csv["cumulativeVaccinated1NumberForDistrict"].sum() != 0:
#             state_raw_csv.loc[state_raw_csv["District"] == district,"deltaVaccinated1ForDistrict"]=state_raw_csv.loc[state_raw_csv["District"] == district,"cumulativeVaccinated1NumberForDistrict"].item()-previous_state_raw_csv.loc[previous_state_raw_csv["District"] == district,"cumulativeVaccinated1NumberForDistrict"].item()

#         if state_raw_csv["cumulativeVaccinated2NumberForDistrict"].sum() != 0:
#             state_raw_csv.loc[state_raw_csv["District"] == district,"deltaVaccinated2ForDistrict"]=state_raw_csv.loc[state_raw_csv["District"] == district,"cumulativeVaccinated2NumberForDistrict"].item()-previous_state_raw_csv.loc[previous_state_raw_csv["District"] == district,"cumulativeVaccinated2NumberForDistrict"].item()
           
    lst = ["Recovered","Confirmed","Deceased","Tested","Vaccinated1","Vaccinated2","Vaccinated"]
    for val in lst:
        state_raw_csv["delta{}ForDistrict".format(val)] = 0
        for idx in state_raw_csv.index:
            # print(new_state_raw_csv["District"][idx])
            # print(new_state_raw_csv["7DmaVaccinatedForDistrict"][idx])
            avg_counter = 0
            value = 0
            i = 1
            prev_date=datetime.strptime(Date,"%Y-%m-%d")+timedelta(days=(-1*i))
            prev_date_str=str(prev_date.date())
            if i == 0:
                previous_state_raw_csv = state_raw_csv
            else:
                previous_state_raw_csv_vacc=pd.read_csv(f"../RAWCSV/{prev_date_str}/{StateCode}_final.csv")
                try:
                    previous_state_raw_csv=pd.read_csv(f"../RAWCSV/{prev_date_str}/{StateCode}_raw.csv")
                except:
                    previous_state_raw_csv=pd.read_csv(f"../RAWCSV/{prev_date_str}/myGov/{StateCode}_raw.csv")
                
            cowin_district_code=cowin_codes.loc[((cowin_codes["district_name"]==state_raw_csv["District"][idx])&(cowin_codes["state_name"]==state_name)),"district_id"]
            cowin_district_code.reset_index(inplace=True,drop=True)
            try:
                cowin_district_code=cowin_district_code[0]
                district_vaccination_data=True
            except IndexError:
                # print(f"{district} not found in Cowin States Districts Code")
                district_vaccination_data=False
                pass
            except KeyError:
                # print(f"{district} not found in Cowin States Districts Code")
                district_vaccination_data=False
                pass
            #Read final file for vaccination
            
            if val in ["Vaccinated1","Vaccinated2"]:
                # if district_vaccination_data:
                #     value = state_raw_csv["cumulative{}NumberForDistrict".format(val)][idx] - vaccination_numbers_yesterday[3][str(cowin_district_code)][["Vaccinated1","Vaccinated2","Vaccinated"].index(val)]
                
                value = state_raw_csv["cumulative{}NumberForDistrict".format(val)][idx] - previous_state_raw_csv_vacc.loc[previous_state_raw_csv_vacc["District"] == state_raw_csv["District"][idx],"cumulative{}NumberForDistrict".format(val)]
            elif val != "Vaccinated":    
                value = state_raw_csv["cumulative{}NumberForDistrict".format(val)][idx] - previous_state_raw_csv.loc[previous_state_raw_csv["District"] == state_raw_csv["District"][idx],"cumulative{}NumberForDistrict".format(val)]
            try:
                state_raw_csv["delta{}ForDistrict".format(val)][idx] = value
            except:
                state_raw_csv["delta{}ForDistrict".format(val)][idx] = None
    
    lst = ["Recovered","Confirmed","Deceased","Tested","Vaccinated1","Vaccinated2"]
#     for val in lst:
#         state_raw_csv["7Dma{}ForDistrict".format(val)] = 0
#         for idx in state_raw_csv.index:
#             # print(new_state_raw_csv["District"][idx])
#             # print(new_state_raw_csv["7DmaVaccinatedForDistrict"][idx])
#             avg_counter = 0
#             value = 0
#             for i in range(7):
#                 prev_date=datetime.strptime(Date,"%Y-%m-%d")+timedelta(days=(-1*i))
#                 prev_date_str=str(prev_date.date())
#                 if i == 0:
#                     previous_state_raw_csv = state_raw_csv
#                 else:
#                     try:
#                         previous_state_raw_csv=pd.read_csv(f"../RAWCSV/{prev_date_str}/{StateCode}_raw.csv")
#                     except:
#                         previous_state_raw_csv=pd.read_csv(f"../RAWCSV/{prev_date_str}/myGov/{StateCode}_raw.csv")
                
#                 cowin_district_code=cowin_codes.loc[((cowin_codes["district_name"]==state_raw_csv["District"][idx])&(cowin_codes["state_name"]==state_name)),"district_id"]
#                 cowin_district_code.reset_index(inplace=True,drop=True)
#                 try:
#                     cowin_district_code=cowin_district_code[0]
#                     district_vaccination_data=True
#                 except IndexError:
#                     # print(f"{district} not found in Cowin States Districts Code")
#                     district_vaccination_data=False
#                     pass
#                 except KeyError:
#                     # print(f"{district} not found in Cowin States Districts Code")
#                     district_vaccination_data=False
#                     pass
#                 else:
#                     ls_districts_add=cowin_codes_map[str(cowin_district_code)]

#                 if val in ["Vaccinated1","Vaccinated2","Vaccinated"]:
#                     if district_vaccination_data:
#                         value += vaccination_numbers[3][str(cowin_district_code)][["Vaccinated1","Vaccinated2","Vaccinated"].index(val)]
#                 else:    
#                     value += previous_state_raw_csv.loc[previous_state_raw_csv["District"] == state_raw_csv["District"][idx],"cumulative{}NumberForDistrict".format(val)]
#                 avg_counter += 1
#             try:
#                 state_raw_csv["7Dma{}ForDistrict".format(val)][idx] = value/7
#             except:
#                 state_raw_csv["7Dma{}ForDistrict".format(val)][idx] = None

    state_raw_csv.to_csv(f"../RAWCSV/{Date}/{StateCode}_final.csv",index=False)
    print (f"Running 7DMA for {StateCode}")
    get_7dma_state(StateCode, Date)
    
        # return(state_raw_csv)

# Run Block

In [11]:
# import pandas as pd
# sources = pd.read_csv("../sources.csv")
# # updateDerivedValues("AS","2021-10-29")
# sources

runDate = "2022-01-13"
removeLogging(runDate)
for key,val in STATE_NAMES.items():
    updateDerivedValues(key,runDate)
    
# 7DMA is being called for "TT" separately since TT.py 7DMA calculations are not correct 
get_7dma_state('TT', runDate)



Andhra Pradesh
---------------------------------
Other States population numbers not found
Other Countries population numbers not found


/home/swiadmin/Incovid19/incovid19/env/lib/python3.6/site-packages/ipykernel_launcher.py:263: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/swiadmin/Incovid19/incovid19/env/lib/python3.6/site-packages/ipykernel_launcher.py:265: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Running 7DMA for AP


Arunachal Pradesh
---------------------------------
Kra Daadi population numbers not found
Lepa Rada population numbers not found
Pakke Kessang population numbers not found
Running 7DMA for AR


Assam
---------------------------------
Airport Quarantine population numbers not found
Charaideo population numbers not found
Other State population numbers not found
Unknown population numbers not found
Running 7DMA for AS


Bihar
---------------------------------


/home/swiadmin/Incovid19/incovid19/srcDerivedValues/get7DMA.py:78: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['7DmaTestedForDistrict'][df['District'] != 'Unknown'] = test_df['7DmaTestedForDistrict'][test_df['District'] != 'Unknown']


Running 7DMA for BR


Chhattisgarh
---------------------------------
Gaurela Pendra Marwahi population numbers not found
Other State population numbers not found
Running 7DMA for CT


Goa
---------------------------------
Other State population numbers not found
Unknown population numbers not found
Running 7DMA for GA


Gujarat
---------------------------------
Running 7DMA for GJ


Haryana
---------------------------------
Running 7DMA for HR


Himachal Pradesh
---------------------------------
Running 7DMA for HP


Jharkhand
---------------------------------
Unknown population numbers not found
Running 7DMA for JH


Karnataka
---------------------------------
Other State population numbers not found
Running 7DMA for KA


Kerala
---------------------------------
Running 7DMA for KL


Madhya Pradesh
---------------------------------
Agar Malwa population numbers not found
Niwari population numbers not found
Other Region population numbers not found
Unknown population numbers not found


# Running CSV files

In [49]:
 !python ../CSV_APIs_Code/main_csv.py

../CSV_APIs_Code/main_csv.py:217: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_district["Others"]=""
../CSV_APIs_Code/main_csv.py:218: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_district["State/UTCode"]=df_district["State/UTCode"].apply(lambda val: STATE_NAMES.get(val))
../CSV_APIs_Code/main_csv.py:229: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pa

In [52]:
# def date_range(start, end):
#     r = (end+timedelta(days=1)-start).days
#     return [start+timedelta(days=i) for i in range(r)]
 

# start_date = "2021-12-29"
# end_date = "2021-12-30"
# end = datetime.strptime(end_date, '%Y-%m-%d')
# start = datetime.strptime(start_date, '%Y-%m-%d')
# dateList = date_range(start, end)

# for date in dateList:
#     print(str(date))
#     updateDerivedValues("PB",str(date))

# updateDerivedValues("MH","2022-01-08")
# updateDerivedValues("MH","2022-01-11")
# updateDerivedValues("BR","2022-01-03")
# updateDerivedValues("BR","2022-01-04")
# updateDerivedValues("JK","2021-12-29")
# updateDerivedValues("JK","2021-12-30")
# updateDerivedValues("MH","2022-01-08")
# updateDerivedValues("ML","2022-01-08")
# updateDerivedValues("RJ","2022-01-04")
# # updateDerivedValues("DL","2021-12-30")
# updateDerivedValues("PY","2021-11-12")
# updateDerivedValues("PY","2021-11-13")
# updateDerivedValues("PY","2021-12-05")
# updateDerivedValues("PY","2021-12-06")
# updateDerivedValues("PY","2021-12-08")
# updateDerivedValues("PY","2021-12-09")
# updateDerivedValues("BR","2021-12-16")
# updateDerivedValues("BR","2021-12-17")
# updateDerivedValues("MN","2021-11-07")
# updateDerivedValues("CT","2021-12-08")
# updateDerivedValues("JK","2021-12-08")
# updateDerivedValues("MN","2021-12-08")
# updateDerivedValues("MH","2021-12-29")
# states = ["AR","BR","CT","HP","JK","MN"]
# for state in states:
#     updateDerivedValues(state,"2021-12-20")



Maharashtra
---------------------------------
Other State population numbers not found


/home/swiadmin/Incovid19/incovid19/env/lib/python3.6/site-packages/ipykernel_launcher.py:263: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/swiadmin/Incovid19/incovid19/env/lib/python3.6/site-packages/ipykernel_launcher.py:265: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


Running 7DMA for MH


Maharashtra
---------------------------------
Other State population numbers not found
Running 7DMA for MH


Bihar
---------------------------------
Running 7DMA for BR


Bihar
---------------------------------
Running 7DMA for BR


Jammu and Kashmir
---------------------------------
Running 7DMA for JK


Jammu and Kashmir
---------------------------------
Running 7DMA for JK


In [ ]:
def UpdateDerivedValues(StateCode:str,Date:str,):
    """
    This function 
    1. Calulates the derived columns – Delta, Delta7 (Statewise and Districtwise) ;
    2. Updates district and population data for the state and districts of the state
    3. Updates Cumulative vaccinated numbers for the state
    4. Calculates the derived numbers for the state
    
    Input: RAWCSV file of each state    
    Output: CSV file updated with the above mentioned values
    Parameters:
    StateCode: StateCode of the corresponding State
    Date: Current Date in format "YYYY-MM-DD" type-casted as string
    
    """
    #Reading CSVs
    cowin_codes=pd.read_csv("CSV/CowinAppStateAndDistrictCode.csv")
#     sources=pd.read_csv("Sources.csv")
#     state_districts=pd.read_csv("StateDistricts.csv")
    state_raw_csv=pd.read_csv(f"RAWCSV/{Date}/{StateCode}_raw.csv",index_col=False)
    state_population=pd.read_csv("CSV/StatePopulation.csv")
    district_population=pd.read_csv("CSV/DistrictPopulation.csv")
    
    
    #Check and insert cummulativeDeceasedState 
    if state_raw_csv["cumulativeDeceasedNumberForState"].sum() == 0:
        if state_raw_csv["cumulativeDeceasedNumberForDistrict"].sum() != 0:
            state_raw_csv["cumulativeDeceasedNumberForState"] = state_raw_csv["cumulativeDeceasedNumberForDistrict"].sum()
    
    #Check for columns in state_raw_csv
    
    state_raw_csv=col_check_state_raw_csv(state_raw_csv)
#     print(state_raw_csv.head())
    
    #Reading the cumulative vaccinated numbers for the state using API
    
    state_name=STATE_NAMES[StateCode]
    print("\n")
    print(state_name)
    print("---------------------------------")
    
     #Add state/ut code
    state_raw_csv["State/UTCode"]=StateCode
    
    
    vaccination_numbers=vaccination_numbers_api(state_name,Date)
    
    state_cumulative_vaccinated1=vaccination_numbers[0]
    state_cumulative_vaccinated2=vaccination_numbers[1]
    state_cumulative_vaccinated=vaccination_numbers[2]
    
    state_raw_csv.loc[state_raw_csv["State/UTCode"]==StateCode,"cumulativeVaccinated1NumberForState"]=state_cumulative_vaccinated1
    state_raw_csv.loc[state_raw_csv["State/UTCode"]==StateCode,"cumulativeVaccinated2NumberForState"]=state_cumulative_vaccinated2
    state_raw_csv.loc[state_raw_csv["State/UTCode"]==StateCode,"cumulativeVaccinatedNumberForState"]=state_cumulative_vaccinated
    
    #Reading State population numbers from static source
    state_population_number=state_population.loc[state_population["State"]==StateCode,"StatePop"]
    state_population_number.reset_index(inplace=True,drop=True)
    state_population_number=state_population_number[0]    
    state_raw_csv.loc[state_raw_csv["State/UTCode"]==StateCode,"statePopulation"]=state_population_number
    
    #Reading the cumulative vaccinated numbers for the districts of the state
    districts=[district for district in state_raw_csv["District"]]
    # print(districts)
    for district in districts:
        print(district,state_name)
        cowin_district_code=cowin_codes.loc[((cowin_codes["district_name"]==district)&(cowin_codes["state_name"]==state_name)),"district_id"]
        cowin_district_code.reset_index(inplace=True,drop=True)
        try:
            cowin_district_code=cowin_district_code[0]
        except IndexError:
            print(f"{district} not found in Cowin States Districts Code")
            district_vaccination_data=False
            pass
        except KeyError:
            print(f"{district} not found in Cowin States Districts Code")
            district_vaccination_data=False
            pass
        else:
            ls_districts_add=cowin_codes_map[str(cowin_district_code)]
            district_cumulative_vaccinated1=0
            district_cumulative_vaccinated2=0
            district_cumulative_vaccinated=0
            for dist in ls_districts_add:
                district_cumulative_vaccinated1+=vaccination_numbers[3][dist][0]
                district_cumulative_vaccinated2+=vaccination_numbers[3][dist][1]
                district_cumulative_vaccinated+=vaccination_numbers[3][dist][2]
                district_vaccination_data=True
            
            #Reading District population numbers from static source
            district_population_number=district_population.loc[((district_population["State"]==StateCode) & (district_population["District"]==district)) ,"DistrictPop"]#[0]
            district_population_number.reset_index(inplace=True,drop=True)
            try:
                district_population_number=district_population_number[0]
            except IndexError:
                print(f"{district} population numbers not found")
                pass
            except KeyError:
                print(f"{district} population numbers not found")
                pass
            else:
                state_raw_csv.loc[((state_raw_csv["State/UTCode"]==StateCode)&(state_raw_csv["District"]==district)),"districtPopulation"]=district_population_number
            
            state_raw_csv.loc[((state_raw_csv["State/UTCode"]==StateCode)&(state_raw_csv["District"]==district)),"cumulativeVaccinated1NumberForDistrict"]=district_cumulative_vaccinated1
            state_raw_csv.loc[((state_raw_csv["State/UTCode"]==StateCode)&(state_raw_csv["District"]==district)),"cumulativeVaccinated2NumberForDistrict"]=district_cumulative_vaccinated2
            state_raw_csv.loc[((state_raw_csv["State/UTCode"]==StateCode)&(state_raw_csv["District"]==district)),"cumulativeVaccinatedNumberForDistrict"]=district_cumulative_vaccinated

        #Deriving the delta numbers
        
        prev_date=datetime.strptime(Date,"%Y-%m-%d")+timedelta(days=(-1))
        prev_date_str=str(prev_date.year)+"-"+str(prev_date.month)+"-"+str(prev_date.day)
        previous_state_raw_csv=pd.read_csv(f"RAWCSV/{prev_date_str}/{StateCode}_raw.csv")
        
        
        # print(type(state_raw_csv["District"]),state_raw_csv.columns)
        
        # for district in state_raw_csv["District"]:
        # print(district)
        # print(state_raw_csv.loc[state_raw_csv["District"] == district,"cumulativeConfirmedNumberForDistrict"].item())
        # print(8*"*")
        # print(previous_state_raw_csv.loc[previous_state_raw_csv["District"] == district,"cumulativeConfirmedNumberForDistrict"])
        # print(8*"*")
        # print(state_raw_csv.loc[state_raw_csv["District"] == district,"cumulativeConfirmedNumberForDistrict"].item()-previous_state_raw_csv.loc[previous_state_raw_csv["District"] == district,"cumulativeConfirmedNumberForDistrict"].item())
        if state_raw_csv["cumulativeConfirmedNumberForDistrict"].sum() != 0:
            state_raw_csv.loc[state_raw_csv["District"] == district,"deltaConfirmedForDistrict"]=state_raw_csv.loc[state_raw_csv["District"] == district,"cumulativeConfirmedNumberForDistrict"].item()-previous_state_raw_csv.loc[previous_state_raw_csv["District"] == district,"cumulativeConfirmedNumberForDistrict"].item()

        if state_raw_csv["cumulativeDeceasedNumberForDistrict"].sum() != 0:
            state_raw_csv.loc[state_raw_csv["District"] == district,"deltaDeceasedForDistrict"]=state_raw_csv.loc[state_raw_csv["District"] == district,"cumulativeDeceasedNumberForDistrict"].item()-previous_state_raw_csv.loc[previous_state_raw_csv["District"] == district,"cumulativeDeceasedNumberForDistrict"].item()

        if state_raw_csv["cumulativeRecoveredNumberForDistrict"].sum() != 0:
            state_raw_csv.loc[state_raw_csv["District"] == district,"deltaRecoveredForDistrict"]=state_raw_csv.loc[state_raw_csv["District"] == district,"cumulativeRecoveredNumberForDistrict"].item()-previous_state_raw_csv.loc[previous_state_raw_csv["District"]  == district,"cumulativeRecoveredNumberForDistrict"].item()

        if state_raw_csv["cumulativeTestedNumberForDistrict"].sum() != 0:
            state_raw_csv.loc[state_raw_csv["District"] == district,"deltaTestedForDistrict"]=state_raw_csv.loc[state_raw_csv["District"] == district,"cumulativeTestedNumberForDistrict"].item()-previous_state_raw_csv.loc[previous_state_raw_csv["District"] == district,"cumulativeTestedNumberForDistrict"].item()

        if state_raw_csv["cumulativeVaccinatedNumberForDistrict"].sum() != 0:
            state_raw_csv.loc[state_raw_csv["District"] == district,"deltaVaccinatedForDistrict"]=state_raw_csv.loc[state_raw_csv["District"] == district,"cumulativeVaccinatedNumberForDistrict"].item()-previous_state_raw_csv.loc[previous_state_raw_csv["District"] == district,"cumulativeVaccinatedNumberForDistrict"].item()

        if state_raw_csv["cumulativeVaccinated1NumberForDistrict"].sum() != 0:
            state_raw_csv.loc[state_raw_csv["District"] == district,"deltaVaccinated1ForDistrict"]=state_raw_csv.loc[state_raw_csv["District"] == district,"cumulativeVaccinated1NumberForDistrict"].item()-previous_state_raw_csv.loc[previous_state_raw_csv["District"] == district,"cumulativeVaccinated1NumberForDistrict"].item()

        if state_raw_csv["cumulativeVaccinated2NumberForDistrict"].sum() != 0:
            state_raw_csv.loc[state_raw_csv["District"] == district,"deltaVaccinated2ForDistrict"]=state_raw_csv.loc[state_raw_csv["District"] == district,"cumulativeVaccinated2NumberForDistrict"].item()-previous_state_raw_csv.loc[previous_state_raw_csv["District"] == district,"cumulativeVaccinated2NumberForDistrict"].item()
            
                
        if state_raw_csv["cumulativeConfirmedNumberForState"].sum() != 0:
            # print(state_raw_csv["cumulativeConfirmedNumberForState"],type(state_raw_csv["cumulativeConfirmedNumberForState"]))
            state_raw_csv["deltaConfirmedForState"]=state_raw_csv["cumulativeConfirmedNumberForState"]-previous_state_raw_csv["cumulativeConfirmedNumberForState"]
        
        if state_raw_csv["cumulativeDeceasedNumberForState"].sum() != 0:
            state_raw_csv["deltaDeceasedForState"]=state_raw_csv["cumulativeDeceasedNumberForState"]-previous_state_raw_csv["cumulativeDeceasedNumberForState"]
        
        if state_raw_csv["cumulativeRecoveredNumberForState"].sum() != 0:
            state_raw_csv["deltaRecoveredForState"]=state_raw_csv["cumulativeRecoveredNumberForState"]-previous_state_raw_csv["cumulativeRecoveredNumberForState"]
        
        if state_raw_csv["cumulativeVaccinatedNumberForState"].sum() != 0:
            # print("today")
            # print(state_raw_csv["cumulativeVaccinatedNumberForState"])
            # print("yesterday")
            # print(previous_state_raw_csv["cumulativeVaccinatedNumberForState"])
            # print("Difference")
            # print(state_raw_csv["cumulativeVaccinatedNumberForState"]-previous_state_raw_csv["cumulativeVaccinatedNumberForState"])
            state_raw_csv["deltaVaccinatedForState"]=state_raw_csv["cumulativeVaccinatedNumberForState"][0]-previous_state_raw_csv["cumulativeVaccinatedNumberForState"][0]
        
        state_raw_csv["deltaVaccinated1ForState"]=state_raw_csv["cumulativeVaccinated1NumberForState"]-previous_state_raw_csv["cumulativeVaccinated1NumberForState"]
        state_raw_csv["deltaVaccinated2ForState"]=state_raw_csv["cumulativeVaccinated2NumberForState"]-previous_state_raw_csv["cumulativeVaccinated2NumberForState"]
        
        
        if state_raw_csv["cumulativeTestedNumberForState"].sum() != 0:
            state_raw_csv["deltaTestedForState"]=state_raw_csv["cumulativeTestedNumberForState"]-previous_state_raw_csv["cumulativeTestedNumberForState"]
        
       
        
        if district_vaccination_data:
            state_raw_csv["deltaVaccinatedForDistrict"]=state_raw_csv["cumulativeVaccinatedNumberForDistrict"]-previous_state_raw_csv["cumulativeVaccinatedNumberForDistrict"]
        avg_counter=1
        
        state_raw_csv["delta7ConfirmedForState"]=0
        state_raw_csv["delta7DeceasedForState"]=0
        state_raw_csv["delta7RecoveredForState"]=0
        state_raw_csv["delta7VaccinatedForState"]=0
        state_raw_csv["delta7TestedForState"]=0
        
        state_raw_csv["delta7ConfirmedForDistrict"]=0
        state_raw_csv["delta7DeceasedForDistrict"]=0
        state_raw_csv["delta7RecoveredForDistrict"]=0
        state_raw_csv["delta7VaccinatedForDistrict"]=0
        state_raw_csv["delta7TestedForState"]=0
        
        #Logic
        #delta7:# on (today) - # of (today-7)
        #delta21_14_: # on (today-21) - # of (today-14)
        #7dma: {(# of today)+(# of today-1)+(# of today-2)+(# of today-3)+(# of today-4)+(# of today-5+# of today-6)}/7
        
        prev_date7=datetime.strptime(Date,"%Y-%m-%d")+timedelta(days=-7)
        prev_date_str=str(prev_date7.year)+"-"+str(prev_date7.month)+"-"+str(prev_date7.day)
        previous_state_raw_csv=pd.read_csv(f"RAWCSV/{prev_date_str}/{StateCode}_raw.csv")
        
        # print(state_raw_csv["cumulativeConfirmedNumberForState"],type(state_raw_csv["cumulativeConfirmedNumberForState"]))
        state_raw_csv["delta7ConfirmedForState"]=state_raw_csv["cumulativeConfirmedNumberForState"]-previous_state_raw_csv["cumulativeConfirmedNumberForState"]
        state_raw_csv["delta7DeceasedForState"]=state_raw_csv["cumulativeDeceasedNumberForState"]-previous_state_raw_csv["cumulativeDeceasedNumberForState"]
        state_raw_csv["delta7RecoveredForState"]=state_raw_csv["cumulativeRecoveredNumberForState"]-previous_state_raw_csv["cumulativeRecoveredNumberForState"]
        state_raw_csv["delta7VaccinatedForState"]=state_raw_csv["cumulativeVaccinatedNumberForState"]-previous_state_raw_csv["cumulativeVaccinatedNumberForState"]
        state_raw_csv["delta7TestedForState"]=state_raw_csv["cumulativeTestedNumberForState"]-previous_state_raw_csv["cumulativeTestedNumberForState"]

        state_raw_csv["delta7ConfirmedForDistrict"]=state_raw_csv["cumulativeConfirmedNumberForDistrict"]-previous_state_raw_csv["cumulativeConfirmedNumberForDistrict"]
        state_raw_csv["delta7DeceasedForDistrict"]=state_raw_csv["cumulativeDeceasedNumberForDistrict"]-previous_state_raw_csv["cumulativeDeceasedNumberForDistrict"]
        state_raw_csv["delta7RecoveredForDistrict"]=state_raw_csv["cumulativeRecoveredNumberForDistrict"]-previous_state_raw_csv["cumulativeRecoveredNumberForDistrict"]
        state_raw_csv["delta7VaccinatedForDistrict"]=state_raw_csv["cumulativeVaccinatedNumberForDistrict"]-previous_state_raw_csv["cumulativeVaccinatedNumberForDistrict"]
        state_raw_csv["delta7VaccinatedForDistrict"]=state_raw_csv["cumulativeTestedNumberForDistrict"]-previous_state_raw_csv["cumulativeTestedNumberForDistrict"]

        
        prev_date21=datetime.strptime(Date,"%Y-%m-%d")+timedelta(days=-21)
        prev_date_21_str=prev_date21.strftime("%Y-%m-%d")
#         str(prev_date21.year)+"-"+str(prev_date21.month)+"-"+str(prev_date21.day)
        previous_state_21_raw_csv=pd.read_csv(f"RAWCSV/{prev_date_21_str}/{StateCode}_raw.csv")
        
        prev_date14=datetime.strptime(Date,"%Y-%m-%d")+timedelta(days=-14)
        prev_date_14_str=str(prev_date14.year)+"-"+str(prev_date14.month)+"-"+str(prev_date14.day)
        previous_state_14_raw_csv=pd.read_csv(f"RAWCSV/{prev_date_14_str}/{StateCode}_raw.csv")
        
        state_raw_csv["delta21_14ConfirmedForState"]=previous_state_14_raw_csv["cumulativeConfirmedNumberForState"]-previous_state_21_raw_csv["cumulativeConfirmedNumberForState"]
        
        # for district in state_raw_csv["District"]:
        state_raw_csv.loc[state_raw_csv["District"] == district,"delta21_14ConfirmedForDistrict"]=previous_state_14_raw_csv.loc[previous_state_14_raw_csv["District"] == district,"cumulativeConfirmedNumberForDistrict"].item()-previous_state_21_raw_csv.loc[previous_state_21_raw_csv["District"] == district,"cumulativeConfirmedNumberForDistrict"].item()
                
        state_raw_csv.loc[:,"7DmaConfirmedForState"]=0
        state_raw_csv.loc[:,"7DmaDeceasedForState"]=0
        state_raw_csv.loc[:,"7DmaRecoveredForState"]=0
        state_raw_csv.loc[:,"7DmaVaccinatedForState"]=0
        state_raw_csv.loc[:,"7DmaTestedForState"]=0
        state_raw_csv.loc[:,"7DmaVaccinated1ForState"]=0
        state_raw_csv.loc[:,"7DmaVaccinated2ForState"]=0

        state_raw_csv.loc[:,"7DmaConfirmedForDistrict"]=0
        state_raw_csv.loc[:,"7DmaDeceasedForDistrict"]=0
        state_raw_csv.loc[:,"7DmaRecoveredForDistrict"]=0
        state_raw_csv.loc[:,"7DmaVaccinatedForDistrict"]=0
        state_raw_csv.loc[:,"7DmaTestedForDistrict"]=0
        state_raw_csv.loc[:,"7DmaVaccinated1ForDistrict"]=0
        state_raw_csv.loc[:,"7DmaVaccinated2ForDistrict"]=0
        x=0
        
        
        avg_counter = 0
        for i in range(7):
            prev_date=datetime.strptime(Date,"%Y-%m-%d")+timedelta(days=(-1*i))
            print(str(prev_date))
            prev_date_str=str(prev_date.year)+"-"+str(prev_date.month)+"-"+str(prev_date.day)
            previous_state_raw_csv=pd.read_csv(f"RAWCSV/{prev_date_str}/{StateCode}_raw.csv")
            
            state_raw_csv["7DmaConfirmedForState"]+=previous_state_raw_csv["cumulativeConfirmedNumberForState"]
            state_raw_csv["7DmaDeceasedForState"]+=previous_state_raw_csv["cumulativeDeceasedNumberForState"]
            state_raw_csv["7DmaRecoveredForState"]+=previous_state_raw_csv["cumulativeRecoveredNumberForState"]
            state_raw_csv["7DmaVaccinatedForState"]+=previous_state_raw_csv["cumulativeVaccinatedNumberForState"]
            state_raw_csv["7DmaTestedForState"]+=previous_state_raw_csv["cumulativeTestedNumberForState"]
            state_raw_csv["7DmaVaccinated1ForState"]+=previous_state_raw_csv["cumulativeVaccinated1NumberForState"]
            state_raw_csv["7DmaVaccinated2ForState"]+=previous_state_raw_csv["cumulativeVaccinated2NumberForState"]
            
            
#             x=x+state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaVaccinatedForDistrict"].item() + previous_state_raw_csv.loc[previous_state_raw_csv["District"] == district,"cumulativeVaccinatedNumberForDistrict"].item()
#             y=previous_state_raw_csv.loc[previous_state_raw_csv["District"] == district,"cumulativeVaccinatedNumberForDistrict"].item()
#             print(x,y)
#             print(x+y)
            avg_counter+=1
        # state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaVaccinatedForDistrict"] = x
        # print(type(state_raw_csv.loc[state_raw_csv.loc[state_raw_csv["District"] == district].index.item(),"7DmaVaccinatedForDistrict"]))
        # state_raw_csv.loc[state_raw_csv.loc[state_raw_csv["District"] == district].index.item(),"7DmaVaccinatedForDistrict"] = float(x)
        # test.loc[test.loc[test["max_speed"] == 4,"shield"].index.item(),"shield"]
        # print(state_raw_csv["7DmaVaccinatedForDistrict"])# = x
#             # for district in state_raw_csv["District"]:
#             # state_raw_csv["7DmaVaccinatedForDistrict"] = state_raw_csv["7DmaVaccinatedForDistrict"].astype('float')
#             state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaConfirmedForDistrict"] = state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaConfirmedForDistrict"].item() + previous_state_raw_csv.loc[previous_state_raw_csv["District"] == district,"cumulativeConfirmedNumberForDistrict"].item()
#             state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaDeceasedForDistrict"] = state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaDeceasedForDistrict"].item() + previous_state_raw_csv.loc[previous_state_raw_csv["District"] == district,"cumulativeDeceasedNumberForDistrict"].item()
#             state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaRecoveredForDistrict"] = state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaRecoveredForDistrict"].item() + previous_state_raw_csv.loc[previous_state_raw_csv["District"] == district,"cumulativeRecoveredNumberForDistrict"].item()
#             state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaVaccinatedForDistrict"][state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaVaccinatedForDistrict"].index] = state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaVaccinatedForDistrict"].item() + previous_state_raw_csv.loc[previous_state_raw_csv["District"] == district,"cumulativeVaccinatedNumberForDistrict"].item()
#             print("District Vac")
#             print(state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaConfirmedForDistrict"].index)
#             print(previous_state_raw_csv.loc[previous_state_raw_csv["District"] == district,"cumulativeVaccinatedNumberForDistrict"])
#             print("Vac Total")
#             print(state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaVaccinatedForDistrict"])
#             state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaTestedForDistrict"] = state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaTestedForDistrict"].item() + previous_state_raw_csv.loc[previous_state_raw_csv["District"] == district,"cumulativeTestedNumberForDistrict"].item()
#             state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaVaccinated1ForDistrict"] = state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaVaccinated1ForDistrict"].item()+previous_state_raw_csv.loc[previous_state_raw_csv["District"] == district,"cumulativeVaccinated1NumberForDistrict"].item()
#             state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaVaccinated2ForDistrict"] = state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaVaccinated2ForDistrict"].item() + previous_state_raw_csv.loc[previous_state_raw_csv["District"] == district,"cumulativeVaccinated2NumberForDistrict"].item()

            
            
        state_raw_csv["7DmaConfirmedForState"]=round(state_raw_csv["7DmaConfirmedForState"]/avg_counter,0)
        state_raw_csv["7DmaDeceasedForState"]=round(state_raw_csv["7DmaDeceasedForState"]/avg_counter,0)
        state_raw_csv["7DmaRecoveredForState"]=round(state_raw_csv["7DmaRecoveredForState"]/avg_counter,0)
        state_raw_csv["7DmaVaccinatedForState"]=round(state_raw_csv["7DmaVaccinatedForState"]/avg_counter,0)
        state_raw_csv["7DmaTestedForState"]=round(state_raw_csv["7DmaTestedForState"]/avg_counter,0)
        state_raw_csv["7DmaVaccinated1ForState"]=round(state_raw_csv["7DmaVaccinated1ForState"]/avg_counter,0)
        state_raw_csv["7DmaVaccinated2ForState"]=round(state_raw_csv["7DmaVaccinated2ForState"]/avg_counter,0)
        
        
        
#         state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaConfirmedForDistrict"]=round(state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaConfirmedForDistrict"].item()/avg_counter,0)
#         state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaDeceasedForDistrict"]=round(state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaDeceasedForDistrict"].item()/avg_counter,0)
#         state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaRecoveredForDistrict"]=round(state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaRecoveredForDistrict"].item()/avg_counter,0)
#         # if state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaVaccinatedForDistrict"].isna().item():
#         #     state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaVaccinatedForDistrict"] = 0
#         # else:
#         # state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaVaccinatedForDistrict"] = round(state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaVaccinatedForDistrict"].item()/avg_counter,0)
#         state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaTestedForDistrict"]=round(state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaTestedForDistrict"].item()/avg_counter,0)
#         state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaVaccinated1ForDistrict"]=round(state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaVaccinated1ForDistrict"].item()/avg_counter,0)
#         state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaVaccinated2ForDistrict"]=round(state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaVaccinated2ForDistrict"].item()/avg_counter,0)
        
        # state_raw_csv["7DmaVaccinatedForDistrict"] = round(state_raw_csv["7DmaVaccinatedForDistrict"]/avg_counter,0)
        
        
#         print("Vac Avg")
#         print(round(state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaVaccinatedForDistrict"]/avg_counter,0))
#         print(10*"&")
#         print(state_raw_csv["7DmaVaccinatedForDistrict"])



        # if state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaVaccinatedForDistrict"].isna().item():
        #     print("True")
            # print(int(round(state_raw_csv.loc[state_raw_csv["District"] == district,"7DmaVaccinatedForDistrict"].item()/avg_counter,0)))
        
#         state_raw_csv["delta21_14ConfirmedForState"]=previous_state_raw_csv["cumulativeConfirmedNumberForState"]
        
#         state_raw_csv["delta14ConfirmedForState"]=0
#         state_raw_csv["delta21ConfirmedForState"]=0
        
#         for i in range(8,15):
#             prev_date=datetime.strptime(Date,"%Y-%m-%d")+timedelta(days=(-1*i))
#             prev_date_str=str(prev_date.year)+"-"+str(prev_date.month)+"-"+str(prev_date.day)
#             previous_state_raw_csv=pd.read_csv(f"./RAWCSV/{prev_date_str}/{StateCode}_raw.csv")
           
#             state_raw_csv["delta14ConfirmedForState"]+=previous_state_raw_csv["cumulativeConfirmedNumberForState"]
            
#         state_raw_csv["delta21ConfirmedForState"]=state_raw_csv["delta14ConfirmedForState"]
#         for i in range(15,22):
#             prev_date=datetime.strptime(Date,"%Y-%m-%d")+timedelta(days=(-1*i))
#             prev_date_str=prev_date.strftime('%Y-%m-%d')
# #             print(prev_date,prev_date_str)
#             previous_state_raw_csv=pd.read_csv(f"./RAWCSV/{prev_date_str}/{StateCode}_raw.csv")
#             state_raw_csv["delta21ConfirmedForState"]+=previous_state_raw_csv["cumulativeConfirmedNumberForState"]
#         state_raw_csv["delta21_14ConfirmedForState"]=state_raw_csv["delta21ConfirmedForState"]-state_raw_csv["delta14ConfirmedForState"]                
    
    lst = ["Recovered","Confirmed","Deceased","Tested","Vaccinated1","Vaccinated2"]
    for val in lst:
        for idx in state_raw_csv.index:
            # print(new_state_raw_csv["District"][idx])
            # print(new_state_raw_csv["7DmaVaccinatedForDistrict"][idx])
            avg_counter = 0
            value = 0
            for i in range(1,8):
                prev_date=datetime.strptime(Date,"%Y-%m-%d")+timedelta(days=(-1*i))
                prev_date_str=str(prev_date.year)+"-"+str(prev_date.month)+"-"+str(prev_date.day)
                previous_state_raw_csv=pd.read_csv(f"RAWCSV/{prev_date_str}/{StateCode}_raw.csv")
                value += previous_state_raw_csv.loc[previous_state_raw_csv["District"] == state_raw_csv["District"][idx],"cumulative{}NumberForDistrict".format(val)]
                avg_counter += 1
            state_raw_csv["7Dma{}ForDistrict".format(val)][idx] = value.item()/avg_counter

        state_raw_csv.to_csv(f"RAWCSV/{Date}/{StateCode}_final.csv",index=False)
        return(state_raw_csv)
            

In [ ]:
# keys=["AR","BR","CT","HP","MN","RJ"]
keys = ["RJ"]
for key in keys:
    new_state_raw_csv=UpdateDerivedValues(key,"2021-10-27")

In [ ]:
Date = "2021-10-27"
StateCode = "RJ"
new_state_raw_csv
lst = ["Recovered","Confirmed","Deceased","Tested","Vaccinated1","Vaccinated2"]
for val in lst:
    for idx in new_state_raw_csv.index:
        # print(new_state_raw_csv["District"][idx])
        # print(new_state_raw_csv["7DmaVaccinatedForDistrict"][idx])
        avg_counter = 0
        value = 0
        for i in range(1,8):
            prev_date=datetime.strptime(Date,"%Y-%m-%d")+timedelta(days=(-1*i))
            prev_date_str=str(prev_date.year)+"-"+str(prev_date.month)+"-"+str(prev_date.day)
            previous_state_raw_csv=pd.read_csv(f"RAWCSV/{prev_date_str}/{StateCode}_raw.csv")
            value += previous_state_raw_csv.loc[previous_state_raw_csv["District"] == new_state_raw_csv["District"][idx],"cumulative{}NumberForDistrict".format(val)]
            avg_counter += 1
        new_state_raw_csv["7Dma{}ForDistrict".format(val)][idx] = value.item()/avg_counter

In [ ]:
new_state_raw_csv["7DmaVaccinated1ForDistrict"]

In [ ]:
import pandas as pd
state_raw_csv=pd.read_csv(f"RAWCSV/2021-10-27/RJ_raw.csv",index_col=False)

In [ ]:
state_raw_csv.columns

In [ ]:
if state_raw_csv["cumulativeDeceasedNumberForState"].sum() == 0:
    if state_raw_csv["cumulativeDeceasedNumberForDistrict"].sum() != 0:
        state_raw_csv["cumulativeDeceasedNumberForState"] = state_raw_csv["cumulativeDeceasedNumberForDistrict"].sum()

In [ ]:
state_raw_csv

In [ ]:
cowin_codes.loc[((cowin_codes["district_name"]==district)&(cowin_codes["state_name"]==state_name)),"district_id"]

In [ ]:
cowin_codes.loc[((cowin_codes["state_name"]=="Andhra Pradesh"))]

In [ ]:
Ananthapur

In [ ]:
previous_state_raw_csv=pd.read_csv(f"RAWCSV/2021-10-26/RJ_raw.csv")

In [ ]:
previous_state_raw_csv

In [ ]:
# state_raw_csv["cumulativeVaccinatedNumberForState"]
previous_state_raw_csv["cumulativeVaccinatedNumberForState"]

In [ ]:
test = pd.DataFrame([[1, 2], [4, 5], [7, 8]],
     index=['cobra', 'viper', 'sidewinder'],
     columns=['max_speed', 'shield'])

In [ ]:
type(test.loc['cobra', 'shield'])

In [ ]:
test.loc[test["max_speed"] == 4,"shield"].index.item()

In [ ]:
test.loc[test.loc[test["max_speed"] == 4,"shield"].index.item(),"shield"]

In [ ]:
test